In [1]:
key = "{AzureLanguageServiceKey}"
endpoint = "{AzureLanguageServiceEndpoint}"



In [2]:
from azure.ai.textanalytics import TextAnalyticsClient
from azure.core.credentials import AzureKeyCredential

# Authenticate the client using your key and endpoint 
def authenticate_client():
    ta_credential = AzureKeyCredential(key)
    text_analytics_client = TextAnalyticsClient(
            endpoint=endpoint, 
            credential=ta_credential)
    return text_analytics_client

client = authenticate_client()

# Example method for detecting sensitive information (PII) from text 
def pii_recognition_example(client):
    documents = [
        "The employee's SSN is 859-98-0987.",
        "The employee's phone number is 555-555-5555."
    ]
    response = client.recognize_pii_entities(documents, language="en")
    result = [doc for doc in response if not doc.is_error]
    for doc in result:
        print("Redacted Text: {}".format(doc.redacted_text))
        for entity in doc.entities:
            print("Entity: {}".format(entity.text))
            print("\tCategory: {}".format(entity.category))
            print("\tConfidence Score: {}".format(entity.confidence_score))
            print("\tOffset: {}".format(entity.offset))
            print("\tLength: {}".format(entity.length))
pii_recognition_example(client)

Redacted Text: The ********'s SSN is ***********.
Entity: employee
	Category: PersonType
	Confidence Score: 0.99
	Offset: 4
	Length: 8
Entity: 859-98-0987
	Category: USSocialSecurityNumber
	Confidence Score: 0.85
	Offset: 22
	Length: 11
Redacted Text: The ********'s phone number is ************.
Entity: employee
	Category: PersonType
	Confidence Score: 0.99
	Offset: 4
	Length: 8
Entity: 555-555-5555
	Category: PhoneNumber
	Confidence Score: 0.8
	Offset: 31
	Length: 12


In [3]:
from azure.ai.textanalytics import TextAnalyticsClient
from azure.core.credentials import AzureKeyCredential

def authenticate_client():
    ta_credential = AzureKeyCredential(key)
    text_analytics_client = TextAnalyticsClient(
            endpoint=endpoint,
            credential=ta_credential)
    return text_analytics_client

def pii_redaction_example(client, input_text):
    # Detecting PII entities in the text
    response = client.recognize_pii_entities(documents=[input_text])[0]

    # Redacting PII from the text
    redacted_text = input_text
    for entity in response.entities:
        redacted_text = redacted_text.replace(entity.text, "***")

    return redacted_text

# Authenticate the client
client = authenticate_client()

# Example text with PII
example_text = "My name is Gosia Borzecka and my phone number is 555-1234. I live in Nottingham on the address 123 Redmond Rd."

# Redacting PII
redacted = pii_redaction_example(client, example_text)
print("Original Text:", example_text)
print("Redacted Text:", redacted)


Original Text: My name is Gosia Borzecka and my phone number is 555-1234. I live in Nottingham on the address 123 Redmond Rd.
Redacted Text: My name is *** and my phone number is ***. I live in Nottingham on the address ***


In [4]:
from openai import OpenAI

client = OpenAI(
    # defaults to os.environ.get("OPENAI_API_KEY")
    api_key="{OpenAIKey}",
)

text = "My name is Gosia Borzecka and my phone number is 555-1234. I live in Nottingham on the address 123 Redmond Rd."

completion = client.chat.completions.create(
    model="gpt-4",
    messages = [
        {"role": "system", "content": "You are an AI language model trained to get protect PII."},
        {"role": "user", "content": f"If you found in text PII, return PII text with '***': {text}"}
    ]
)

print(text)
print(completion.choices[0].message.content)
print("***")

My name is Gosia Borzecka and my phone number is 555-1234. I live in Nottingham on the address 123 Redmond Rd.
My name is *** and my phone number is ***. I live in *** on the address ***.
***
